# roastcoffea: Performance Monitoring for Coffea

This notebook shows how to track performance metrics in your Coffea analysis.

In [34]:
from contextlib import contextmanager

from coffea import processor
from coffea.nanoevents import NanoAODSchema
from dask.distributed import Client, LocalCluster

from roastcoffea import MetricsCollector, track_memory, track_metrics, track_time


@contextmanager
def acquire_client(n_workers=2, threads_per_worker=1):
    """Context manager for Dask client."""
    cluster = LocalCluster(
        n_workers=n_workers,
        threads_per_worker=threads_per_worker,
        processes=True,
    )
    client = Client(cluster)
    print(f"Dashboard: {client.dashboard_link}")

    try:
        yield client
    finally:
        client.close()
        cluster.close()

## Basic Metrics Collection

Wrap your workflow with `MetricsCollector` to track overall performance.

In [35]:
# Simple processor
class MyProcessor(processor.ProcessorABC):
    def process(self, events):
        jets = events.Jet
        selected = jets[jets.pt > 30]
        return {"sum": len(events), "njets": len(selected)}

    def postprocess(self, accumulator):
        return accumulator

In [36]:
fileset = {
    "ttbar": {
        "files": {
            "root://eospublic.cern.ch//eos/opendata/cms/mc/RunIISummer20UL16NanoAODv9/TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8/NANOAODSIM/106X_mcRun2_asymptotic_v17-v1/120000/08FCB2ED-176B-064B-85AB-37B898773B98.root": "Events"
        }
    }
}

In [ ]:
my_processor = MyProcessor()

with acquire_client() as client:
    with MetricsCollector(client, processor_instance=my_processor) as collector:
        executor = processor.DaskExecutor(client=client)
        runner = processor.Runner(
            executor=executor,
            schema=NanoAODSchema,
            chunksize=100_000,
            savemetrics=True,
        )

        output, report = runner(
            fileset, processor_instance=my_processor, treename="Events"
        )
        collector.set_coffea_report(report)

    collector.print_summary()

## Chunk-Level Tracking

Add `@track_metrics` to see per-chunk performance.

In [38]:
class DetailedProcessor(processor.ProcessorABC):
    @track_metrics
    def process(self, events):
        jets = events.Jet
        selected = jets[jets.pt > 30]
        return {"sum": len(events), "njets": len(selected)}

    def postprocess(self, accumulator):
        return accumulator

In [39]:
detailed_processor = DetailedProcessor()

with acquire_client() as client:
    with MetricsCollector(client, processor_instance=detailed_processor) as collector:
        executor = processor.DaskExecutor(client=client)
        runner = processor.Runner(
            executor=executor,
            schema=NanoAODSchema,
            chunksize=100_000,
            savemetrics=True,
        )

        output, report = runner(
            fileset, processor_instance=detailed_processor, treename="Events"
        )

        # Extract chunk metrics from output
        collector.extract_metrics_from_output(output)
        collector.set_coffea_report(report)

    collector.print_summary()

/Users/moaly/Work/iris-hep/roastcoffea/.pixi/envs/dev/lib/python3.13/site-packages/distributed/node.py:187: 
UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 53837 instead
  warnings.warn(

Dashboard: http://127.0.0.1:53837/status

/Users/moaly/Work/iris-hep/roastcoffea/.pixi/envs/dev/lib/python3.13/site-packages/coffea/nanoevents/schemas/nanoaod.py:264: RuntimeWarning: Missing cross-reference index for LowPtElectron_electronIdx => Electron
  warnings.warn(
/Users/moaly/Work/iris-hep/roastcoffea/.pixi/envs/dev/lib/python3.13/site-packages/coffea/nanoevents/schemas/nanoaod.py:264: RuntimeWarning: Missing cross-reference index for LowPtElectron_photonIdx => Photon
  warnings.warn(
/Users/moaly/Work/iris-hep/roastcoffea/.pixi/envs/dev/lib/python3.13/site-packages/coffea/nanoevents/schemas/nanoaod.py:264: RuntimeWarning: Missing cross-reference index for LowPtElectron_electronIdx => Electron
  warnings.warn(
/Users/moaly/Work/iris-hep/roastcoffea/.pixi/envs/dev/lib/python3.13/site-packages/coffea/nanoevents/schemas/nanoaod.py:264: RuntimeWarning: Missing cross-reference index for LowPtElectron_photonIdx => Photon
  warnings.warn(


Processing 100% ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14/14 [ 0:00:24 < 0:00:00 | 0.5 chunk/s ]

                 Throughput Metrics                 
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┓
┃ Metric                    ┃ Value                ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━┩
│ Data Rate                 │ 0.01 Gbps (1.7 MB/s) │
│ Total Bytes Read (Coffea) │ 38.78 MB             │
│ Memory Read (Dask Spans)  │ 39.58 KB             │
└───────────────────────────┴──────────────────────┘

            Event Processing Metrics            
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Metric                     ┃ Value           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ Total Events               │ 1,233,000       │
│ Event Rate (Wall Clock)    │ 50.6 kHz        │
│ Event Rate (Aggregated)    │ 79.7 kHz        │
│ Event Rate (Core-Averaged) │ 25297.7 Hz/core │
│ Efficiency Ratio           │ 63.5%           │
└────────────────────────────┴─────────────────┘

          Resource Utilization          
┏━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━┓
┃ Metric                   ┃ Value     ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━┩
│ Workers (Time-Averaged)  │ 2.0       │
│ Peak Workers             │ 2         │
│ Cores per Worker         │ 1.0       │
│ Total Cores              │ 2         │
│ Core Efficiency          │ 31.7%     │
│ Speedup Factor           │ 0.6x      │
│ Peak Memory (per worker) │ 365.44 MB │
│ Avg Memory (per worker)  │ 276.02 MB │
└──────────────────────────┴───────────┘

       Timing Breakdown       
┏━━━━━━━━━━━━━━━━━━━━┳━━━━━━━┓
┃ Metric             ┃ Value ┃
┡━━━━━━━━━━━━━━━━━━━━╇━━━━━━━┩
│ Wall Time          │ 24.4s │
│ Total CPU Time     │ 15.5s │
│ Number of Chunks   │ 12    │
│ Avg CPU Time/Chunk │ 1.3s  │
└────────────────────┴───────┘

    Fine Metrics (from Dask Spans)    
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━┓
┃ Metric                     ┃ Value ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━┩
│ Processor CPU Time         │ 8.6s  │
│ Processor Non-CPU Time     │ 38.6s │
│   CPU %                    │ 18.3% │
│   Non-CPU %                │ 81.7% │
│ Dask Overhead CPU Time     │ 0.0s  │
│ Dask Overhead Non-CPU Time │ 0.0s  │
└────────────────────────────┴───────┘

         Chunk Metrics          
┏━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━┓
┃ Metric            ┃ Value    ┃
┡━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━┩
│ Total Chunks      │ 12       │
│ Mean Chunk Time   │ 1.3s     │
│   • Min           │ 0.4s     │
│   • Max           │ 2.9s     │
│   • Std Dev       │ 0.7s     │
│ Mean Memory Delta │ 25.6 MB  │
│   • Min           │ -13.9 MB │
│   • Max           │ 49.0 MB  │
│ Mean Events/Chunk │ 102750   │
│   • Min           │ 102750   │
│   • Max           │ 102750   │
└───────────────────┴──────────┘

In [40]:
# Access individual chunk metrics
metrics = collector.get_metrics()
print(f"Total chunks: {metrics['num_chunks']}")
print(f"Average chunk time: {metrics['chunk_duration_mean']:.2f}s")

Total chunks: 12

Average chunk time: 1.29s

## Fine-Grained Profiling

Track specific sections of your analysis.

In [ ]:
import awkward as ak


class ProfilingProcessor(processor.ProcessorABC):
    @track_metrics
    def process(self, events):
        with track_time(self, "load_jets"):
            jets = events.Jet

        with track_memory(self, "selection"):
            selected = jets[jets.pt > 30]

        ak.sum(selected.pt, axis=1)  # Force evaluation for accurate profiling

        return {"sum": len(events), "njets": len(selected)}

    def postprocess(self, accumulator):
        return accumulator

In [46]:
profiling_processor = ProfilingProcessor()

with acquire_client() as client:
    with MetricsCollector(client, processor_instance=profiling_processor) as collector:
        executor = processor.DaskExecutor(client=client)
        runner = processor.Runner(
            executor=executor,
            schema=NanoAODSchema,
            chunksize=100_000,
            savemetrics=True,
        )

        output, report = runner(
            fileset, processor_instance=profiling_processor, treename="Events"
        )
        collector.extract_metrics_from_output(output)
        collector.set_coffea_report(report)

    collector.print_summary()

/Users/moaly/Work/iris-hep/roastcoffea/.pixi/envs/dev/lib/python3.13/site-packages/distributed/node.py:187: 
UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 53944 instead
  warnings.warn(

Dashboard: http://127.0.0.1:53944/status

/Users/moaly/Work/iris-hep/roastcoffea/.pixi/envs/dev/lib/python3.13/site-packages/coffea/nanoevents/schemas/nanoaod.py:264: RuntimeWarning: Missing cross-reference index for LowPtElectron_electronIdx => Electron
  warnings.warn(
/Users/moaly/Work/iris-hep/roastcoffea/.pixi/envs/dev/lib/python3.13/site-packages/coffea/nanoevents/schemas/nanoaod.py:264: RuntimeWarning: Missing cross-reference index for LowPtElectron_photonIdx => Photon
  warnings.warn(
/Users/moaly/Work/iris-hep/roastcoffea/.pixi/envs/dev/lib/python3.13/site-packages/coffea/nanoevents/schemas/nanoaod.py:264: RuntimeWarning: Missing cross-reference index for LowPtElectron_electronIdx => Electron
  warnings.warn(
/Users/moaly/Work/iris-hep/roastcoffea/.pixi/envs/dev/lib/python3.13/site-packages/coffea/nanoevents/schemas/nanoaod.py:264: RuntimeWarning: Missing cross-reference index for LowPtElectron_photonIdx => Photon
  warnings.warn(


Processing 100% ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14/14 [ 0:00:31 < 0:00:00 | 0.4 chunk/s ]

                 Throughput Metrics                 
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┓
┃ Metric                    ┃ Value                ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━┩
│ Data Rate                 │ 0.01 Gbps (1.3 MB/s) │
│ Total Bytes Read (Coffea) │ 38.78 MB             │
│ Memory Read (Dask Spans)  │ 21.63 KB             │
└───────────────────────────┴──────────────────────┘

            Event Processing Metrics            
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Metric                     ┃ Value           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ Total Events               │ 1,233,000       │
│ Event Rate (Wall Clock)    │ 39.6 kHz        │
│ Event Rate (Aggregated)    │ 47.9 kHz        │
│ Event Rate (Core-Averaged) │ 19803.7 Hz/core │
│ Efficiency Ratio           │ 82.7%           │
└────────────────────────────┴─────────────────┘

          Resource Utilization          
┏━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━┓
┃ Metric                   ┃ Value     ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━┩
│ Workers (Time-Averaged)  │ 2.0       │
│ Peak Workers             │ 2         │
│ Cores per Worker         │ 1.0       │
│ Total Cores              │ 2         │
│ Core Efficiency          │ 41.3%     │
│ Speedup Factor           │ 0.8x      │
│ Peak Memory (per worker) │ 340.23 MB │
│ Avg Memory (per worker)  │ 282.28 MB │
└──────────────────────────┴───────────┘

       Timing Breakdown       
┏━━━━━━━━━━━━━━━━━━━━┳━━━━━━━┓
┃ Metric             ┃ Value ┃
┡━━━━━━━━━━━━━━━━━━━━╇━━━━━━━┩
│ Wall Time          │ 31.1s │
│ Total CPU Time     │ 25.7s │
│ Number of Chunks   │ 12    │
│ Avg CPU Time/Chunk │ 2.1s  │
└────────────────────┴───────┘

    Fine Metrics (from Dask Spans)    
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━┓
┃ Metric                     ┃ Value ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━┩
│ Processor CPU Time         │ 7.7s  │
│ Processor Non-CPU Time     │ 46.5s │
│   CPU %                    │ 14.3% │
│   Non-CPU %                │ 85.7% │
│ Dask Overhead CPU Time     │ 0.0s  │
│ Dask Overhead Non-CPU Time │ 0.0s  │
└────────────────────────────┴───────┘

     Chunk Metrics      
┏━━━━━━━━━━━━━━┳━━━━━━━┓
┃ Metric       ┃ Value ┃
┡━━━━━━━━━━━━━━╇━━━━━━━┩
│ Total Chunks │ 12    │
└──────────────┴───────┘

In [ ]:
# Look at section timing
for chunk in collector.chunk_metrics[:3]:  # First 3 chunks
    print(f"\nChunk with {chunk['num_events']} events:")
    for section, duration in chunk["timing"].items():
        print(f"  {section}: {duration:.3f}s")
    for section, delta_mb in chunk["memory"].items():
        print(f"  {section}: {delta_mb:+.1f} MB")

## Saving Results

Save metrics for later analysis.

In [ ]:
from pathlib import Path

# Save measurement
output_dir = Path("measurements")
measurement_path = collector.save_measurement(
    output_dir, measurement_name="ttbar_analysis"
)
print(f"Saved to: {measurement_path}")

In [ ]:
# Load it back
from roastcoffea.export.measurements import load_measurement

loaded = load_measurement(measurement_path)
print(f"Loaded metrics: {loaded['metrics']['total_events']} events processed")